In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
sales_train = pd.read_csv('sales_train.csv')
shops = pd.read_csv('shops.csv')
items = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
test = pd.read_csv('test.csv')

In [3]:
test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [4]:
sales_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [5]:
#Filtering sales for October 2015 using boolean
#October 2015 has value 33 in date_block_num column
sales_oct = sales_train.date_block_num==33

In [6]:
sales_oct

0          False
1          False
2          False
3          False
4          False
           ...  
2935844     True
2935845     True
2935846     True
2935847     True
2935848     True
Name: date_block_num, Length: 2935849, dtype: bool

In [7]:
#Making a subset of values that are True from filtering 
#Rows that are True are removed from the main dataset
sales_train_oct = sales_train[sales_oct]

In [8]:
sales_train_oct

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2882335,23.10.2015,33,45,13315,649.0,1.0
2882336,05.10.2015,33,45,13880,229.0,1.0
2882337,02.10.2015,33,45,13881,659.0,1.0
2882338,12.10.2015,33,45,13881,659.0,1.0
2882339,04.10.2015,33,45,13923,169.0,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.0,1.0
2935845,09.10.2015,33,25,7460,299.0,1.0
2935846,14.10.2015,33,25,7459,349.0,1.0
2935847,22.10.2015,33,25,7440,299.0,1.0


In [9]:
#Confirming that subset is smaller
len(sales_train), len(sales_train_oct), len(sales_oct)

(2935849, 53514, 2935849)

In [10]:
#compairing the subset to test, there are only two relevant columns, shop and item id
#I also need item_cnt_day to get the total number of item sold in a month
#Therefore I only need 3 out of 6 columns in the subset(sales_train_oct)
#I will then group these 3 columns by the columns in the test file (shop_id and item_id)
#I will also need to find the sum of item_cnt_day using agg()
group = sales_train_oct[['shop_id','item_id','item_cnt_day']].groupby(['shop_id','item_id']).agg({'item_cnt_day':'sum'}).reset_index()

#since I found the sum of item_cnt_day in the dataset 
#this means it is now item count in the month of Oct not daily anymore, so I renamed it
group = group.rename(columns={'item_cnt_day':'item_cnt_month'})
group 

,shop_id,item_id,item_cnt_month
0,2,31,1.0
1,2,486,3.0
2,2,787,1.0
3,2,794,1.0
4,2,968,1.0
...,...,...,...
31526,59,22087,6.0
31527,59,22088,2.0
31528,59,22091,1.0
31529,59,22100,1.0


In [11]:
#test has shop_id and item_id columns but I want to add item_cnt_month in group to it 
#so I merged shop_id and item_id columns in test and group
#the new test will have item_cnt_month because group has it
test = pd.merge(test,group, on = ['shop_id','item_id'], how = 'left')
test 

,ID,shop_id,item_id,item_cnt_month
0,0,5,5037,NaN
1,1,5,5320,NaN
2,2,5,5233,1.0
3,3,5,5232,NaN
4,4,5,5268,NaN
...,...,...,...,...
214195,214195,45,18454,1.0
214196,214196,45,16188,NaN
214197,214197,45,15757,NaN
214198,214198,45,19648,NaN


In [12]:
#Replacing NaN values in column item_cnt_month with 0 and clipping it
test['item_cnt_month'] = test.item_cnt_month.fillna(0).clip(lower=0,upper=20)
test.head(10)

,ID,shop_id,item_id,item_cnt_month
0,0,5,5037,0.0
1,1,5,5320,0.0
2,2,5,5233,1.0
3,3,5,5232,0.0
4,4,5,5268,0.0
5,5,5,5039,1.0
6,6,5,5041,2.0
7,7,5,5046,0.0
8,8,5,5319,0.0
9,9,5,5003,0.0


In [13]:
#Reducing test to only two columns
test = test[['ID','item_cnt_month']]
test.head(10)

,ID,item_cnt_month
0,0,0.0
1,1,0.0
2,2,1.0
3,3,0.0
4,4,0.0
5,5,1.0
6,6,2.0
7,7,0.0
8,8,0.0
9,9,0.0


In [14]:
#Setting index as column ID
submission = test.set_index('ID')
submission.head(10) 

,item_cnt_month
ID,
0,0.0
1,0.0
2,1.0
3,0.0
4,0.0
5,1.0
6,2.0
7,0.0
8,0.0


In [15]:
#Converting to csv file
submission.to_csv('submission_sales.csv')